# Amazon Knowledge Base Connection Test
Este notebook permite probar la conexión con Amazon Knowledge Bases utilizando las credenciales definidas en el archivo `.env`.

In [ ]:
import os
import sys
import asyncio
import json
import boto3
from dotenv import load_dotenv

current_dir = os.getcwd()
src_path = os.path.join(current_dir, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

load_dotenv()

# Mapear credenciales para boto3 si están en minúsculas en el .env
for key in ["aws_access_key_id", "aws_secret_access_key", "aws_region", "aws_session_token"]:
    val = os.getenv(key)
    if val:
        os.environ[key.upper()] = val

from src.config import settings
from src.adapter.AmazonKnowledgeBaseProvider import AmazonKnowledgeBaseProvider
from src.core.models import QueryRequest

print(f"✅ Región configurada en Settings: {settings.AWS_REGION}")
print(f"✅ Perfil configurado: {settings.AWS_PROFILE}")

✅ Región configurada en Settings: us-east-1
✅ Perfil configurado: None


## 1. Verificar Identidad y Región Real
Vamos a ver quién somos y en qué región está operando boto3 por defecto.

In [2]:
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    session = boto3.Session()
    print(f"✅ Conectado como: {identity['Arn']}")
    print(f"✅ Región de la sesión: {session.region_name}")
except Exception as e:
    print(f"❌ Error de AWS: {e}")

✅ Conectado como: arn:aws:iam::846588355404:user/admin-user-05112025
✅ Región de la sesión: us-east-1


## 2. Listar KBs y obtener su Región
Buscamos la KB y extraemos su ARN para estar 100% seguros de su región.

In [3]:
provider = AmazonKnowledgeBaseProvider()
client = boto3.client('bedrock-agent', region_name=settings.AWS_REGION)

async def get_all_kbs_debug():
    print(f"Listando KBs en la región {settings.AWS_REGION}...")
    summaries = client.list_knowledge_bases().get('knowledgeBaseSummaries', [])
    
    if not summaries:
        print(f"⚠️ No se encontraron KBs en {settings.AWS_REGION}. ¿Es esta la región correcta?")
        return []

    results = []
    for kb in summaries:
        kb_id = kb['knowledgeBaseId']
        # Obtener detalles para ver el ARN
        details = client.get_knowledge_base(knowledgeBaseId=kb_id).get('knowledgeBase', {})
        arn = details.get('knowledgeBaseArn', 'N/A')
        print(f"- {kb['name']} | ID: {kb_id} | ARN: {arn}")
        results.append(kb_id)
    return results

all_ids = await get_all_kbs_debug()

Listando KBs en la región us-east-1...
- kb-demo02-data-automation | ID: SKSSB4G0M7 | ARN: arn:aws:bedrock:us-east-1:846588355404:knowledge-base/SKSSB4G0M7
- kb-demo01-bsaravia | ID: IPJ5LJA6LQ | ARN: arn:aws:bedrock:us-east-1:846588355404:knowledge-base/IPJ5LJA6LQ


## 3. Probar Consulta
Si el ID existe pero falla la consulta, compararemos la región del ARN con la región de `settings.AWS_REGION`.

In [5]:
async def test_query_safe(kb_id):
    print(f"\nProbando query en KB: {kb_id} usando región {settings.AWS_REGION}...")
    request = QueryRequest(query="test", knowledge_base_id=kb_id, num_results=1)
    try:
        resp = await provider.retrieve(request)
        print("Conexión completa establecida.")
    except Exception as e:
        print(f"❌ Falló la consulta: {e}")
        if "ResourceNotFoundException" in str(e):
            print("\n💡 ANÁLISIS: AWS dice que la KB no existe. Esto ocurre casi siempre porque")
            print(f"la KB está en una región distinta a '{settings.AWS_REGION}'.")
            print("Revisa el ARN arriba; si dice 'us-east-1' pero tu config dice 'us-west-2', deberás cambiar la región en el .env.")

if all_ids:
    await test_query_safe(all_ids[0])


Probando query en KB: SKSSB4G0M7 usando región us-east-1...
Conexión completa establecida.


In [ ]:
from rich import print as rprint
request = QueryRequest(query="que informacion hay en esta base de conocimiento?", knowledge_base_id=all_ids[0], num_results=3)
res = await provider.retrieve(request)
rprint(res)

QueryResponse(
    results=[
        SearchResult(
            content={
                'text': 'COLECCIONES DE PROMPTS     A continuación, se presentan algunas colecciones destacadas de 
prompts:     Biblioteca de prompts de Content Scale26: Esta biblioteca está diseñada para abordar una amplia gama 
de casos de uso y necesidades, dirigida a modelos de IA como ChatGPT, Bard, Midjourney, DALL-E 2, entre otros. 
Permite a los usuarios explorar categorías específicas para obtener respuestas detalladas o iniciar proyectos 
creativos de diversas índoles.     Prompts para la creación de contenido27: Esta colección incluye sugerencias bien
elaboradas por expertos como Amin Boulouma, Robert Brandl y Charlotte Evans. Abarca áreas como ventas y marketing, 
atención al cliente, desarrollo de productos, creación de contenido y redacción publicitaria, proporcionando 
herramientas útiles para mejorar la generación de contenidos.REPORTE DE TECNOLOGÍA: AI GENERATIVA 18     Prompts 
para arte generado por IA27: Para quienes buscan generar arte mediante IA, existen colecciones específicas de 
prompts para plataformas como DALL-E, Midjourney y Stable Diffusion.',
                'type': 'TEXT'
            },
            location={
                's3Location': {
                    'uri': 's3://s3-prueba-docs-08sept2025/docs_demo01/Reporte-de-tecnologia-IA-Generativa.pdf'
                },
                'type': 'S3'
            },
            score=0.6259970366954803
        ),
        SearchResult(
            content={
                'text': 'Aurora: Aurora es la plataforma interna de IA Generativa del Banco Interamericano de 
Desarrollo, diseñada para mejorar la eficiencia en la búsqueda, el resumen y la generación de contenido a partir 
deREPORTE DE TECNOLOGÍA: AI GENERATIVA 14     documentos institucionales. La plataforma alberga múltiples 
asistentes, cada uno de ellos diseñado para acceder a conjuntos específicos de documentos, lo que permite a los 
usuarios enfocar sus consultas y respuestas. Aurora ayuda a los usuarios a encontrar información rápidamente, 
resumir los puntos clave y generar nuevo contenido, todo ello dentro de un entorno seguro.     Seek: Disponible en 
el sitio web de publicaciones del Grupo BID, facilita y acelera el acceso y la utilización del conocimiento dentro 
de nuestras publicaciones, mediante el uso de IA Generativa1. Seek genera una respuesta basada exclusivamente en la
información contenida en la publicación. Además, la respuesta incluye una cita vinculada que permite a los usuarios
navegar directamente a la página relacionada con la respuesta para obtener más detalles.     ATAL: El BID está 
llevando a cabo pruebas con un chatbot de IA Generativa diseñado para facilitar material didáctico en el marco del 
programa «Aprendamos Todos a Leer» (ATAL).',
                'type': 'TEXT'
            },
            location={
                's3Location': {
                    'uri': 's3://s3-prueba-docs-08sept2025/docs_demo01/Reporte-de-tecnologia-IA-Generativa.pdf'
                },
                'type': 'S3'
            },
            score=0.6158619523048401
        ),
        SearchResult(
            content={
                'text': 'aprender y crear contenido nuevo que emula la creatividad humana.     Utilizando 
algoritmos sofisticados y grandes volúmenes de datos, la IA Generativa no solo está transformando la forma en que 
las máquinas interactúan con el mundo, sino también en cómo contribuyen a una gran variedad de sectores, desde la 
salud hasta la eficiencia en la productividad o la creatividad. La habilidad de estos sistemas para generar nuevos 
puntos de datos, textos o soluciones basadas en patrones aprendidos destaca su potencial para revolucionar 
industrias e impulsar la innovación. Dado su crecimiento y expansión, comprender las capacidades y las 
implicaciones de la IA Generativa se ha convertido en un aspecto clave tanto para individuos como para 
organizaciones.     Este informe tien

## 4. Probar consulta y generacion de respuesta 

In [6]:
from rich import print as rprint
request = QueryRequest(query="que informacion hay en esta base de conocimiento?", knowledge_base_id=all_ids[0], num_results=3)
res = await provider.retrieve_and_generate(request)
rprint(res)

QueryResponse(
    results=[
        SearchResult(
            content={
                'text': 'Finanzas_Personales  \t\tFINANZAS PERSONALES (DATOS FICTICIOS)    
\t\tPersona\t\tMes\t\tIngreso\t\tRenta\t\tAlimentación\t\tTransporte\t\tEducación\t\tEntretenimiento\t\tAhorro\t\tD
euda\t\tBalance  \t\tAna Pérez\t\tEnero\t\t1667\t\t563\t\t423\t\t369\t\t188\t\t246\t\t240\t\t46\t\t-408  
\t\t\t\tFebrero\t\t2417\t\t574\t\t402\t\t455\t\t489\t\t175\t\t60\t\t162\t\t100  
\t\t\t\tMarzo\t\t1937\t\t539\t\t219\t\t452\t\t420\t\t515\t\t176\t\t28\t\t-412  
\t\t\t\tAbril\t\t2077\t\t168\t\t541\t\t555\t\t339\t\t143\t\t243\t\t44\t\t44  \t\tLuis 
Gómez\t\tEnero\t\t1450\t\t149\t\t438\t\t573\t\t475\t\t597\t\t294\t\t35\t\t-1111  
\t\t\t\tFebrero\t\t2460\t\t475\t\t191\t\t166\t\t558\t\t123\t\t111\t\t12\t\t824  
\t\t\t\tMarzo\t\t2057\t\t593\t\t393\t\t114\t\t319\t\t390\t\t229\t\t51\t\t-32  
\t\t\t\tAbril\t\t2392\t\t570\t\t374\t\t309\t\t321\t\t188\t\t216\t\t196\t\t218  \t\tCarla 
Torres\t\tEnero\t\t2384\t\t384\t\t425\t\t396\t\t424\t\t549\t\t248\t\t191\t\t-233  
\t\t\t\tFebrero\t\t1609\t\t403\t\t305\t\t379\t\t138\t\t333\t\t164\t\t10\t\t-123  
\t\t\t\tMarzo\t\t1833\t\t225\t\t245\t\t573\t\t147\t\t574\t\t224\t\t141\t\t-296  
\t\t\t\tAbril\t\t1381\t\t580\t\t316\t\t370\t\t259\t\t176\t\t214\t\t150\t\t-684      Inventario_Sucursal  
\t\tINVENTARIO POR SUCURSAL    \t\tSucursal\t\tDirección\t\tProducto\t\tCategoría\t\tStock\t\tStock Mín\t\tCosto 
Unitario\t\tValor Total\t\tObservaciones  \t\tQuito Norte\t\tAv. Amazonas 
123\t\tLaptop\t\tTecnología\t\t18\t\t10\t\t793\t\t14274\t\tOK  
\t\t\t\t\t\tImpresora\t\tTecnología\t\t5\t\t10\t\t533\t\t2665\t\tReponer  
\t\t\t\t\t\tRouter\t\tTecnología\t\t23\t\t10\t\t942\t\t21666\t\tOK  \t\tGuayaquil Centro\t\tMalecón Simón 
Bolívar\t\tLaptop\t\tTecnología\t\t34\t\t10\t\t460\t\t15640\t\tOK  
\t\t\t\t\t\tImpresora\t\tTecnología\t\t24\t\t10\t\t865\t\t20760\t\tOK  
\t\t\t\t\t\tRouter\t\tTecnología\t\t14\t\t10\t\t243\t\t3402\t\tOK',
                'metadata': {
                    'x-amz-bedrock-kb-source-file-modality': 'TEXT',
                    'x-amz-bedrock-kb-data-source-id': 'SBMV6DUDVG'
                }
            },
            location={
                's3Location': {'uri': 's3://s3-prueba-docs-02/excel_nivel_medio_finanzas_inventario.xlsx'},
                'type': 'S3'
            },
            score=None
        ),
        SearchResult(
            content={
                'text': 'Hostil_Mix  \t\tEMPRESA FICTICIA – FINANZAS + INVENTARIOS + SUCURSALES (HOSTIL)      
\t\tSucursal\t\tDirección\t\t\t\tFinanzas\t\t\t\t\t\t\t\t\t\t\t\t\t\tInventario\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tNota
s  
\t\t\t\t\t\t\t\tIngreso\t\tGastos\t\tAhorro\t\tDeuda\t\tBalance\t\t\t\t\t\tProducto\t\tCategoría\t\tStock\t\tMín\t\
tCosto\t\tValor    \t\tSucursal_0\t\tDirección confusa 
#0\t\t\t\t11282\t\t6090\t\t672\t\t125\t\t=\t\t\t\t\t\tLaptop\t\tInfra\t\t30\t\t10\t\t721\t\tN/A\t\t\t\t\t\tTexto 
largo no estructurado con saltos - punto - punto    \t\tSucursal_1\t\tDirección confusa 
#1\t\t\t\t5225\t\t2413\t\t581\t\t424\t\t=\t\t\t\t\t\tLaptop\t\tTech\t\t47\t\t10\t\t401\t\tN/A\t\t\t\t\t\tTexto 
largo no estructurado con saltos - punto - punto    \t\tSucursal_2\t\tDirección confusa 
#2\t\t\t\t11030\t\t7178\t\t2286\t\t1030\t\t=\t\t\t\t\t\tRouter\t\tInfra\t\t29\t\t10\t\t1388\t\tN/A\t\t\t\t\t\tTexto
largo no estructurado con saltos - punto - punto    \t\tSucursal_3\t\tDirección confusa 
#3\t\t\t\t13381\t\t7737\t\t910\t\t818\t\t=\t\t\t\t\t\tLaptop\t\tTech\t\t34\t\t10\t\t1810\t\tN/A\t\t\t\t\t\tTexto 
largo no estructurado con saltos - punto - punto    \t\tSucursal_4\t\tDirección confusa 
#4\t\t\t\t9425\t\t2696\t\t761\t\t382\t\t=\t\t\t\t\t\tCaja\t\tTech\t\t28\t\t10\t\t1977\t\tN/A\t\t\t\t\t\tTexto largo
no estructurado con saltos - punto - punto    \t\tSucursal_5\t\tDirección confusa 
#5\t\t\t\t8900\t\t5247\t\t2958\t\t729\t\t=\t\t\t\t\t\tRouter\t\tTech\t\t8\t\t10\t\t833\t\tN/A\t\t\t\t\t\tTexto 
largo no estructurado con saltos - punto - punto    \t\tSuc

### END